In [14]:
import importlib
import pickle

import keras_tuner as kt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, layers, callbacks
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

import utils
importlib.reload(utils)

from utils import create_dataset, prepare_dataframe

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

In [23]:
DATASET_DIR = "datasets/utkface"
INPUT_SHAPE = (200, 200, 3)

FIXED_KERNEL_SIZE = (3, 3)
MIN_DENSE_UNITS = 16
NUM_AGE_CLASSES = 9

In [17]:
# Only process information for age class <= 8 (80-89) removing 200~ rows

df_full = prepare_dataframe(DATASET_DIR)
df = df_full[df_full['Age_Class'] <= 8].copy()

shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [18]:
df_full

,Image_Path,Age,Gender,Ethnicity,Age_Class
0,datasets/utkface/100_0_0_20170112213500903.jpg...,100,0,0,10
1,datasets/utkface/100_0_0_20170112215240346.jpg...,100,0,0,10
2,datasets/utkface/100_1_0_20170110183726390.jpg...,100,1,0,10
3,datasets/utkface/100_1_0_20170112213001988.jpg...,100,1,0,10
4,datasets/utkface/100_1_0_20170112213303693.jpg...,100,1,0,10
...,...,...,...,...,...
23700,datasets/utkface/9_1_3_20161220222856346.jpg.c...,9,1,3,0
23701,datasets/utkface/9_1_3_20170104222949455.jpg.c...,9,1,3,0
23702,datasets/utkface/9_1_4_20170103200637399.jpg.c...,9,1,4,0
23703,datasets/utkface/9_1_4_20170103200814791.jpg.c...,9,1,4,0


In [19]:
shuffled_df

,Image_Path,Age,Gender,Ethnicity,Age_Class
0,datasets/utkface/35_0_0_20170117191042138.jpg....,35,0,0,3
1,datasets/utkface/4_1_4_20161221193345109.jpg.c...,4,1,4,0
2,datasets/utkface/28_0_3_20170119211807137.jpg....,28,0,3,2
3,datasets/utkface/16_1_0_20170104013358122.jpg....,16,1,0,1
4,datasets/utkface/3_1_0_20170109194247387.jpg.c...,3,1,0,0
...,...,...,...,...,...
23531,datasets/utkface/32_1_0_20170117134737646.jpg....,32,1,0,3
23532,datasets/utkface/68_1_1_20170113012709282.jpg....,68,1,1,6
23533,datasets/utkface/25_1_1_20170112192328981.jpg....,25,1,1,2
23534,datasets/utkface/16_0_0_20170110232714508.jpg....,16,0,0,1


In [20]:
print(shuffled_df['Age_Class'].max()) # 8 is the biggest class
print(shuffled_df['Age_Class'].nunique()) # 9 total classes (0-8)

8
9


In [21]:
# Train-test split, used for evaluating final model
df_train, df_test = train_test_split(shuffled_df, test_size=0.2, random_state=42)

# Train-val split, used for evaluating intermediate model
df_train_tune, df_val_tune = train_test_split(df_train, test_size=0.2, random_state=42)

In [22]:
classes =np.unique(df_train['Age_Class'])
print(f"Classes: {classes}")
weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=df_train['Age_Class']
)
class_weight_dict = dict(zip(classes, weights))
print(f"Class weights calculated for tuner: {class_weight_dict}")

Classes: [0 1 2 3 4 5 6 7 8]
Class weights calculated for tuner: {np.int64(0): np.float64(0.8528332653893193), np.int64(1): np.float64(1.711947626841244), np.int64(2): np.float64(0.35809654228004106), np.int64(3): np.float64(0.5709606986899564), np.int64(4): np.float64(1.1654596100278551), np.int64(5): np.float64(1.1526170798898072), np.int64(6): np.float64(1.9606373008434863), np.int64(7): np.float64(3.729055258467023), np.int64(8): np.float64(5.114914425427873)}


In [ ]:
# Based on existing research found here: https://www.kaggle.com/datasets/jangedoo/utkface-new/code

class AgeHyperModel(kt.HyperModel):
    def __init__(self, use_class_weighting=False):
        self.use_class_weighting = use_class_weighting
    
    def build(self, hp):
        inputs = Input(shape=INPUT_SHAPE, dtype=tf.float32, name="input_image")
        
        hp_use_batch_norm = hp.Boolean("use_batch_norm_global", default=False)
        hp_bn_position = hp.Choice("bn_position", values=["before", "after"], default="before")

        hp_dropout_rate = hp.Float("dropout_rate_global", min_value=0.1, max_value=0.5, step=0.1, default=0.25)
        hp_num_conv_blocks = hp.Int("num_conv_blocks", min_value=1, max_value=4, step=1, default=3)
        
        current_filters = 0
        x = inputs

        for i in range(hp_num_conv_blocks):
            if i == 0:
                current_filters = hp.Choice("filters_start", values=[32, 64], default=32)
            else:
                current_filters = min(512, current_filters * 2)

            x = layers.Conv2D(filters=current_filters, kernel_size=FIXED_KERNEL_SIZE, activation=None)(x)

            if hp_use_batch_norm:
                if hp_bn_position == "before":
                    x = layers.BatchNormalization()(x)
                    x = layers.Activation("relu")(x)
                else:
                    x = layers.Activation("relu")(x)
                    x = layers.BatchNormalization()(x)
            else:
                x = layers.Activation("relu")(x)
                
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        
        last_conv_filters = current_filters
        x = layers.Flatten(name="flatten")(x)

        hp_num_dense_layers = hp.Int("num_dense_layers", min_value=1, max_value=3, step=1, default=2)
        current_dense_units = 0

        for i in range(hp_num_dense_layers):
            hp_size_choice = hp.Choice("size_choice", values=["same", "half"], default="half")

            if hp_size_choice == "same":
                current_dense_units = max(MIN_DENSE_UNITS, last_conv_filters)
            else:
                current_dense_units = max(MIN_DENSE_UNITS, last_conv_filters // 2)

            x = layers.Dense(units=current_dense_units, activation="relu")(x)
            x = layers.Dropout(rate=hp_dropout_rate)(x)
        
        outputs = layers.Dense(NUM_AGE_CLASSES, activation="softmax", name="age_class_output")(x)
        model = Model(inputs=inputs, outputs=outputs)
        
        hp_learning_rate = hp.Choice("learning_rate", values=[0.0001, 0.001, 0.01], default=0.001)
        hp.Choice("batch_size", values=[32, 64, 128, 256], default=64)

        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
        model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

        return model

    # Idea taken from: https://github.com/keras-team/keras-tuner/issues/122#issuecomment-544648268
    def fit(self, hp, model, train_paths, train_labels, validation_data, **kwargs):
        batch_size = hp.Choice("batch_size", values=[32, 64, 128, 256], default=64)

        train_dataset = create_dataset(
            train_paths,
            train_labels,
            batch_size=batch_size,
            is_training=True
        )

        val_paths, val_labels = validation_data
        val_dataset = create_dataset(
            val_paths,
            val_labels,
            batch_size=batch_size,
            is_training=False
        )
        
        fit_kwargs = kwargs

        if self.use_class_weighting:
            print(f"Tuner trial: Applying class weights: {class_weight_dict}")
            fit_kwargs['class_weight'] = class_weight_dict
        else:
             print(f"Tuner trial: Not applying class weights.")

        return model.fit(
            train_dataset,
            validation_data=val_dataset,
            **fit_kwargs,
        )

In [24]:
tuner = kt.Hyperband(
    AgeHyperModel(),
    objective="val_accuracy",
    factor=3,
    directory="hyperparameter_tuning",
    project_name="ageclass_tuning",
)

I0000 00:00:1744479594.750537   17439 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20900 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Reloading Tuner from hyperparameter_tuning/ageclass_tuning/tuner0.json


In [ ]:
tuner.search(
    df_train_tune['Image_Path'],
    df_train_tune['Age_Class'],
    validation_data=(df_val_tune['Image_Path'], df_val_tune['Age_Class']),
    epochs=20,
    callbacks=[callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

In [ ]:
best_hps_list = tuner.get_best_hyperparameters(num_trials=1)
best_hps = best_hps_list[0]

print("Best Hyperparameters Found:")
for hp_name, hp_value in best_hps.values.items():
    print(f"{hp_name}: {hp_value}")

In [ ]:
best_models_list = tuner.get_best_models(num_models=1)
best_model = best_models_list[0]
best_model.summary()

In [ ]:
hps_filename_pkl = "best_hyperparameters.pkl"
with open(hps_filename_pkl, 'wb') as f:
    pickle.dump(best_hps, f)

In [ ]:
with open("models/best_hyperparameters.pkl", 'rb') as f:
    loaded_best_hps = pickle.load(f)

print("Loaded HyperParameters Object:")
print(loaded_best_hps)
print("Values:", loaded_best_hps.values)

hypermodel_instance = tuner.hypermodel
reloaded_model_pkl = hypermodel_instance.build(loaded_best_hps)
reloaded_model_pkl.summary(line_length=100)

Values: {'use_batch_norm_global': False, 'bn_position': 'before', 'dropout_rate_global': 0.2, 'num_conv_blocks': 3, 'filters_start': 64, 'num_dense_layers': 3, 'size_choice': 'half', 'learning_rate': 0.0001, 'batch_size': 64, 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 'tuner/bracket': 4, 'tuner/round': 2, 'tuner/trial_id': '0100'}
Model: "model"
____________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        
 input_image (InputLayer)                    [(None, 200, 200, 3)]                   0              
                                                                                                    
 conv2d (Conv2D)                             (None, 198, 198, 64)                    1792           
                                                                                                    
 activation (Activation)                   

2025-04-12 15:38:22.569427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-12 15:38:23.025425: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31088 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:18:00.0, compute capability: 7.0


In [23]:
COMPILE_ARGS_BASE = {
    'optimizer': tf.keras.optimizers.Adam(learning_rate=loaded_best_hps.get('learning_rate')),
    'loss': 'sparse_categorical_crossentropy',
    'metrics': ['accuracy']
}

FIT_ARGS_BASE = {
    'epochs': 50,
    'batch_size': loaded_best_hps.get('batch_size'),
    'validation_split': 0.0,
    'verbose': 1,
    'callbacks': [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
}

N_SPLITS = 5
BATCH_SIZE = loaded_best_hps.get('batch_size')
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
def run_cross_validation(
    X_paths,
    y_labels,
    best_hps,
    COMPILE_ARGS,
    FIT_ARGS,
    BATCH_SIZE,           
    IMG_HEIGHT=200,           
    IMG_WIDTH=200,          
    N_SPLITS=5,
    RANDOM_STATE=42,
    use_class_weighting=False,
    dataset_name="Dataset"
):
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    fold_no = 1
    fold_scores = {}
    fold_histories = []

    fit_class_weights = None
    local_fit_args = FIT_ARGS

    if use_class_weighting:
        classes = np.unique(y_labels)
        weights = compute_class_weight(
            class_weight='balanced',
            classes=classes,
            y=y_labels
        )
        fit_class_weights = dict(zip(classes, weights))
        local_fit_args['class_weight'] = fit_class_weights


    for train_index, val_index in skf.split(X_paths, y_labels):
        X_train_paths = X_paths[train_index]
        y_train_labels = y_labels[train_index]
        X_val_paths = X_paths[val_index]
        y_val_labels = y_labels[val_index]

        train_ds = create_dataset(
            X_train_paths, y_train_labels,
            batch_size=BATCH_SIZE,
            img_height=IMG_HEIGHT, img_width=IMG_WIDTH,
            is_training=True
        )
        val_ds = create_dataset(
            X_val_paths, y_val_labels,
            batch_size=BATCH_SIZE,
            img_height=IMG_HEIGHT, img_width=IMG_WIDTH,
            is_training=False
        )

        cv_model = hypermodel_instance.build(best_hps)
        cv_model.compile(**COMPILE_ARGS)

        history = cv_model.fit(
            train_ds,
            validation_data=val_ds,
            **local_fit_args
        )
        fold_histories.append(history)

        scores = cv_model.evaluate(val_ds, verbose=0)

        print(f"Fold {fold_no}: Evaluation Results ->")
        for metric_name, score in zip(cv_model.metrics_names, scores):
            print(f"  {metric_name}: {score:.4f}")
            if metric_name not in fold_scores:
                fold_scores[metric_name] = []
            fold_scores[metric_name].append(score)

        fold_no += 1

    print(f"----- Aggregating CV Results for: {dataset_name} -----")
    aggregated_scores = {}

    for metric_name, scores_list in fold_scores.items():
        avg = np.mean(scores_list)
        std = np.std(scores_list)
        aggregated_scores[metric_name] = {'mean': avg, 'std': std}

        print(f"Metric: {metric_name}")
        formatted_scores = [f"{s:.4f}" for s in scores_list]
        print(f"Scores per fold: {formatted_scores}")
        if 'accuracy' in metric_name.lower():
             print(f"  Average: {avg * 100:.2f}%")
             print(f"  Standard Deviation: {std * 100:.2f}%")
        else:
             print(f"  Average: {avg:.4f}")
             print(f"  Standard Deviation: {std:.4f}")
        print("-" * 30)

    return aggregated_scores

In [ ]:
# Cross-Validation Base Dataset

X_base = np.array(df_train["Image_Path"])
y_base = np.array(df_train["Age_Class"])

base_cv_results = run_cross_validation(
    X_paths=X_base,
    y_labels=y_base,
    best_hps=loaded_best_hps,
    COMPILE_ARGS=COMPILE_ARGS_BASE,
    FIT_ARGS=FIT_ARGS_BASE,
    BATCH_SIZE=BATCH_SIZE,
    IMG_HEIGHT=200,
    IMG_WIDTH=200,
    use_class_weighting=False,
    dataset_name="Base Dataset"
)

Epoch 1/50
235/235 [==============================] - 13s 53ms/step - loss: 1.7530 - accuracy: 0.3628 - val_loss: 1.4502 - val_accuracy: 0.4426
Epoch 2/50
235/235 [==============================] - 12s 53ms/step - loss: 1.4677 - accuracy: 0.4408 - val_loss: 1.3678 - val_accuracy: 0.4756
Epoch 3/50
235/235 [==============================] - 12s 53ms/step - loss: 1.3664 - accuracy: 0.4716 - val_loss: 1.2986 - val_accuracy: 0.4931
Epoch 4/50
235/235 [==============================] - 12s 53ms/step - loss: 1.2903 - accuracy: 0.4932 - val_loss: 1.2416 - val_accuracy: 0.5090
Epoch 5/50
235/235 [==============================] - 12s 53ms/step - loss: 1.2200 - accuracy: 0.5150 - val_loss: 1.1956 - val_accuracy: 0.5226
Epoch 6/50
235/235 [==============================] - 12s 53ms/step - loss: 1.1728 - accuracy: 0.5273 - val_loss: 1.1854 - val_accuracy: 0.5305
Epoch 7/50
235/235 [==============================] - 12s 52ms/step - loss: 1.1220 - accuracy: 0.5479 - val_loss: 1.1420 - val_accuracy:

235/235 [==============================] - 12s 53ms/step - loss: 0.8257 - accuracy: 0.6555 - val_loss: 1.1996 - val_accuracy: 0.5381
Epoch 15/50
235/235 [==============================] - 12s 53ms/step - loss: 0.7821 - accuracy: 0.6739 - val_loss: 1.2091 - val_accuracy: 0.5453
Fold 4: Evaluation Results ->
  loss: 1.1289
  accuracy: 0.5511
Epoch 1/50
235/235 [==============================] - 13s 53ms/step - loss: 1.6941 - accuracy: 0.3840 - val_loss: 1.4436 - val_accuracy: 0.4606
Epoch 2/50
235/235 [==============================] - 12s 52ms/step - loss: 1.4238 - accuracy: 0.4589 - val_loss: 1.3515 - val_accuracy: 0.4935
Epoch 3/50
235/235 [==============================] - 12s 53ms/step - loss: 1.3006 - accuracy: 0.4888 - val_loss: 1.2753 - val_accuracy: 0.5068
Epoch 4/50
235/235 [==============================] - 12s 53ms/step - loss: 1.2335 - accuracy: 0.5070 - val_loss: 1.2056 - val_accuracy: 0.5270
Epoch 5/50
235/235 [==============================] - 12s 53ms/step - loss: 1.1656

In [ ]:
# Training Base Dataset

reloaded_model_pkl.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=loaded_best_hps.get('learning_rate')),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

train_ds = create_dataset(
    df_train_tune['Image_Path'],
    df_train_tune['Age_Class'],
    batch_size=loaded_best_hps.get('batch_size'),
    is_training=True
)

val_ds = create_dataset(
    df_val_tune['Image_Path'],
    df_val_tune['Age_Class'],
    batch_size=loaded_best_hps.get('batch_size'),
    is_training=False
)

history = reloaded_model_pkl.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=loaded_best_hps.get('batch_size'),
    callbacks=[callbacks.EarlyStopping(patience=3, restore_best_weights=True)],
    verbose=1
)

Epoch 1/100
235/235 [==============================] - 13s 53ms/step - loss: 1.8509 - accuracy: 0.3359 - val_loss: 1.5575 - val_accuracy: 0.4384
Epoch 2/100
235/235 [==============================] - 12s 53ms/step - loss: 1.5228 - accuracy: 0.4365 - val_loss: 1.3812 - val_accuracy: 0.4764
Epoch 3/100
235/235 [==============================] - 12s 53ms/step - loss: 1.4032 - accuracy: 0.4682 - val_loss: 1.2968 - val_accuracy: 0.4952
Epoch 4/100
235/235 [==============================] - 12s 53ms/step - loss: 1.3180 - accuracy: 0.4856 - val_loss: 1.2493 - val_accuracy: 0.5027
Epoch 5/100
235/235 [==============================] - 12s 53ms/step - loss: 1.2469 - accuracy: 0.5072 - val_loss: 1.2005 - val_accuracy: 0.5074
Epoch 6/100
235/235 [==============================] - 12s 53ms/step - loss: 1.1931 - accuracy: 0.5247 - val_loss: 1.2307 - val_accuracy: 0.5064
Epoch 7/100
235/235 [==============================] - 12s 53ms/step - loss: 1.1442 - accuracy: 0.5457 - val_loss: 1.1587 - val_ac

In [ ]:
reloaded_model_pkl.save("base_model_trained.h5")

<h1 style="font-size:60px; line-height:1.2;">
  Class Weighting Model
</h1>

In [30]:
tuner_balanced = kt.Hyperband(
    AgeHyperModel(use_class_weighting=True),
    objective="val_accuracy",
    factor=3,
    directory="hyperparameter_tuning",
    project_name="ageclass_tuning_balanced",
)

Reloading Tuner from hyperparameter_tuning/ageclass_tuning_balanced/tuner0.json


In [16]:
tuner_balanced.search(
    df_train_tune['Image_Path'],
    df_train_tune['Age_Class'],
    validation_data=(df_val_tune['Image_Path'], df_val_tune['Age_Class']),
    epochs=20,
    callbacks=[callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

Trial 254 Complete [00h 02m 58s]
val_accuracy: 0.40042486786842346

Best val_accuracy So Far: 0.5270844101905823
Total elapsed time: 03h 10m 42s


In [17]:
best_hps_list_balanced = tuner_balanced.get_best_hyperparameters(num_trials=1)
best_hps_balanced = best_hps_list_balanced[0]

print("--- Best Hyperparameters Found ---")
for hp_name, hp_value in best_hps_balanced.values.items():
    print(f"{hp_name}: {hp_value}")

--- Best Hyperparameters Found ---
use_batch_norm_global: False
bn_position: after
dropout_rate_global: 0.5
num_conv_blocks: 4
filters_start: 64
num_dense_layers: 3
size_choice: same
learning_rate: 0.0001
batch_size: 64
tuner/epochs: 34
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0


In [18]:
best_models_list_balanced = tuner_balanced.get_best_models(num_models=1)
best_model_balanced = best_models_list_balanced[0]
best_model_balanced.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 200, 200, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 198, 198, 64)      1792      
                                                                 
 activation (Activation)     (None, 198, 198, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 128)       73856     
                                                                 
 activation_1 (Activation)   (None, 97, 97, 128)       0         
                                                             

In [ ]:
hps_filename_pkl_balanced = "best_hyperparameters_balanced.pkl"
with open(hps_filename_pkl_balanced, 'wb') as f:
    pickle.dump(best_hps_balanced, f)

In [ ]:
with open("models/best_hyperparameters_balanced.pkl", 'rb') as f:
    loaded_best_hps_balanced = pickle.load(f)

print("Loaded HyperParameters Object:")
print(loaded_best_hps_balanced)
print("Values:", loaded_best_hps_balanced.values)

hypermodel_instance = tuner_balanced.hypermodel
reloaded_model_pkl_balanced = hypermodel_instance.build(loaded_best_hps_balanced)
reloaded_model_pkl_balanced.summary(line_length=100)

Loaded HyperParameters Object:
Values: {'use_batch_norm_global': False, 'bn_position': 'after', 'dropout_rate_global': 0.5, 'num_conv_blocks': 4, 'filters_start': 64, 'num_dense_layers': 3, 'size_choice': 'same', 'learning_rate': 0.0001, 'batch_size': 64, 'tuner/epochs': 34, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
Model successfully rebuilt from loaded hyperparameters object:
Model: "model"
____________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        
 input_image (InputLayer)                    [(None, 200, 200, 3)]                   0              
                                                                                                    
 conv2d (Conv2D)                             (None, 198, 198, 64)                    1792           
                                                                             

In [ ]:
# Cross-Validation Class Weighting Dataset

COMPILE_ARGS_BALANCED = {
    'optimizer': tf.keras.optimizers.Adam(learning_rate=loaded_best_hps_balanced.get('learning_rate')),
    'loss': 'sparse_categorical_crossentropy',
    'metrics': ['accuracy']
}

FIT_ARGS_BALANCED = {
    'epochs': 50,
    'batch_size': loaded_best_hps_balanced.get('batch_size'),
    'validation_split': 0.0,
    'verbose': 1,
    'callbacks': [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
}

N_SPLITS = 5
BATCH_SIZE = loaded_best_hps_balanced.get('batch_size')
AUTOTUNE = tf.data.AUTOTUNE

X_base = np.array(df_train["Image_Path"])
y_base = np.array(df_train["Age_Class"])

base_cv_results_balanced = run_cross_validation(
    X_paths=X_base,
    y_labels=y_base,
    best_hps=loaded_best_hps_balanced,
    COMPILE_ARGS=COMPILE_ARGS_BALANCED,
    FIT_ARGS=FIT_ARGS_BALANCED,
    BATCH_SIZE=BATCH_SIZE,
    IMG_HEIGHT=200,
    IMG_WIDTH=200,
    use_class_weighting=True,
    dataset_name="Dataset Class Weighting"
)

Epoch 1/50
235/235 [==============================] - 16s 63ms/step - loss: 2.1996 - accuracy: 0.1012 - val_loss: 2.1944 - val_accuracy: 0.1275
Epoch 2/50
235/235 [==============================] - 15s 62ms/step - loss: 2.1321 - accuracy: 0.1816 - val_loss: 1.8566 - val_accuracy: 0.3279
Epoch 3/50
235/235 [==============================] - 15s 62ms/step - loss: 1.8147 - accuracy: 0.3095 - val_loss: 1.5735 - val_accuracy: 0.3901
Epoch 4/50
235/235 [==============================] - 15s 62ms/step - loss: 1.6264 - accuracy: 0.3551 - val_loss: 1.4845 - val_accuracy: 0.4063
Epoch 5/50
235/235 [==============================] - 15s 62ms/step - loss: 1.5206 - accuracy: 0.3887 - val_loss: 1.3703 - val_accuracy: 0.4538
Epoch 6/50
235/235 [==============================] - 15s 62ms/step - loss: 1.4431 - accuracy: 0.4103 - val_loss: 1.3565 - val_accuracy: 0.4644
Epoch 7/50
235/235 [==============================] - 15s 62ms/step - loss: 1.3811 - accuracy: 0.4301 - val_loss: 1.3150 - val_accuracy:

Epoch 17/50
235/235 [==============================] - 15s 62ms/step - loss: 1.0164 - accuracy: 0.5363 - val_loss: 1.2216 - val_accuracy: 0.5003
Epoch 18/50
235/235 [==============================] - 14s 62ms/step - loss: 0.9827 - accuracy: 0.5363 - val_loss: 1.1581 - val_accuracy: 0.5204
Epoch 19/50
235/235 [==============================] - 15s 62ms/step - loss: 0.9576 - accuracy: 0.5574 - val_loss: 1.2117 - val_accuracy: 0.5029
Epoch 20/50
235/235 [==============================] - 15s 62ms/step - loss: 0.9051 - accuracy: 0.5634 - val_loss: 1.2084 - val_accuracy: 0.5090
Fold 3: Evaluation Results ->
  loss: 1.1374
  accuracy: 0.5356
Epoch 1/50
235/235 [==============================] - 15s 63ms/step - loss: 2.1990 - accuracy: 0.0943 - val_loss: 2.1550 - val_accuracy: 0.1278
Epoch 2/50
235/235 [==============================] - 24s 101ms/step - loss: 2.1879 - accuracy: 0.1688 - val_loss: 2.0763 - val_accuracy: 0.2675
Epoch 3/50
235/235 [==============================] - 15s 63ms/step

In [ ]:
# Training Class Weighting Dataset

reloaded_model_pkl_balanced.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=loaded_best_hps_balanced.get('learning_rate')),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

train_ds_balanced = create_dataset(
    df_train_tune['Image_Path'],
    df_train_tune['Age_Class'],
    batch_size=loaded_best_hps_balanced.get('batch_size'),
    is_training=True
)

val_ds_balanced = create_dataset(
    df_val_tune['Image_Path'],
    df_val_tune['Age_Class'],
    batch_size=loaded_best_hps_balanced.get('batch_size'),
    is_training=False
)

history_balanced = reloaded_model_pkl_balanced.fit(
    train_ds_balanced,
    validation_data=val_ds_balanced,
    epochs=100,
    batch_size=loaded_best_hps_balanced.get('batch_size'),
    class_weight=class_weight_dict,
    callbacks=[callbacks.EarlyStopping(patience=3, restore_best_weights=True)],
    verbose=1
)

Epoch 1/100
235/235 [==============================] - 15s 62ms/step - loss: 1.0647 - accuracy: 0.5864 - val_loss: 1.2175 - val_accuracy: 0.5112
Epoch 2/100
235/235 [==============================] - 14s 61ms/step - loss: 1.0554 - accuracy: 0.5741 - val_loss: 1.2244 - val_accuracy: 0.4995
Epoch 3/100
235/235 [==============================] - 14s 61ms/step - loss: 1.0238 - accuracy: 0.5846 - val_loss: 1.2284 - val_accuracy: 0.5042
Epoch 4/100
235/235 [==============================] - 14s 61ms/step - loss: 1.0116 - accuracy: 0.5858 - val_loss: 1.2342 - val_accuracy: 0.5011


In [ ]:
reloaded_model_pkl_balanced.save("class_weight_model_trained.h5")